# Libraries

In [41]:
import random
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk

# Setup and Preprocessing

In [42]:
# Initialize lemmatizer and download necessary NLTK data
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load the dataset
with open('/content/first-aid-datasets-updated-2.json') as file:
    data = json.load(file)

# Preprocess patterns for training
def preprocess_sentence(sentence):
    # Tokenize, lemmatize, and remove stop words
    tokens = sentence.split()
    tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.lower() not in stop_words]
    return ' '.join(tokens)

patterns = []
responses = []
tags = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        preprocessed_pattern = preprocess_sentence(pattern)
        patterns.append(preprocessed_pattern)
        responses.append(intent['responses'])
        tags.append(intent['tag'])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(patterns, tags, test_size=0.2, random_state=42)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Model Training

In [43]:
# Create and train the machine learning model
model = make_pipeline(
    TfidfVectorizer(max_features=None, ngram_range=(1, 2)),
    LinearSVC()
)
model.fit(X_train, y_train)

# Evaluate model performance with additional metrics
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.2f}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.2f}")
print(f"F1-Score: {f1_score(y_test, y_pred, average='weighted'):.2f}")

Accuracy: 94.87%
Precision: 0.94
Recall: 0.95
F1-Score: 0.94


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



# Logical and Probabilistic Reasoning

In [44]:
# Logical Reasoning Rules
logical_rules = {
    ("fever", "headache"): "You might have the flu. Rest and hydrate.",
    ("cut", "bleeding"): "Apply pressure to stop the bleeding and clean the wound."
}

def logical_reasoning(user_input_tokens):
    """Perform logical reasoning based on input tokens."""
    for rule_conditions, conclusion in logical_rules.items():
        if all(condition in user_input_tokens for condition in rule_conditions):
            return conclusion
    return None

# Probabilistic Reasoning
def bayesian_reasoning(symptoms):
    """Perform probabilistic reasoning using a basic Bayesian model."""
    probabilities = {"fever": 0.8, "headache": 0.7, "nausea": 0.5}
    posterior = 1
    for symptom in symptoms:
        posterior *= probabilities.get(symptom, 0.1)
    return f"Based on symptoms, there is a {posterior * 100:.2f}% chance of illness."

#  Knowledge Representation (Ontology)

In [45]:
ontology = {
    "fever": [],
    "bleeding": [],
    "headache": []
}

def semantic_understanding(user_input):
    """Map user input to known concepts in the ontology."""
    tokens = user_input.split()
    matched_concepts = []
    for concept, keywords in ontology.items():
        if any(keyword in tokens for keyword in keywords):
            matched_concepts.append(concept)
    return matched_concepts

#  Feedback Learning Mechanism

In [46]:
def add_feedback_to_data(user_input, predicted_intent):
    """Add user feedback (new data) to the training set for future learning."""
    new_data = {"patterns": [user_input], "tag": predicted_intent, "responses": ["Thank you for your input!"]}
    data['intents'].append(new_data)  # Append to existing dataset
    patterns.append(preprocess_sentence(user_input))  # Preprocess and add pattern
    tags.append(predicted_intent)  # Add predicted intent tag
    # Retrain model with the updated data
    X_train_new, _, y_train_new, _ = train_test_split(patterns, tags, test_size=0.2, random_state=42)
    model.fit(X_train_new, y_train_new)

# Chatbot Response Logic

In [47]:
# Define a confidence threshold
CONFIDENCE_THRESHOLD = 0.1

def chatbot_response(user_input):
    # Preprocess user input
    preprocessed_input = preprocess_sentence(user_input)

    # Logical reasoning
    user_tokens = preprocessed_input.split()
    logical_result = logical_reasoning(user_tokens)
    if logical_result:
        return logical_result

    # Semantic understanding
    matched_concepts = semantic_understanding(preprocessed_input)
    if matched_concepts:
        return bayesian_reasoning(matched_concepts)

    # Machine learning-based intent prediction
    vectorizer = model.named_steps['tfidfvectorizer']
    classifier = model.named_steps['linearsvc']
    input_vector = vectorizer.transform([preprocessed_input])
    decision_values = classifier.decision_function(input_vector)

    # Determine intent with the highest confidence
    intent_index = decision_values.argmax()
    confidence = decision_values[0][intent_index]
    predicted_intent = classifier.classes_[intent_index]

    # Respond based on confidence
    if confidence >= CONFIDENCE_THRESHOLD:
        for intent_data in data['intents']:
            if intent_data['tag'] == predicted_intent:
                return random.choice(intent_data['responses'])
    else:
        # If the confidence is low, ask for feedback and add it to the training data
        add_feedback_to_data(user_input, predicted_intent)
        return "I'm sorry, I couldn't understand that. Could you rephrase?"

# Chat Loop

In [ ]:
print("Hello! I am your first aid assistant. Type 'quit' to exit.")

while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        print("Goodbye!")
        break

    response = chatbot_response(user_input)
    print(f"Bot: {response}")

Hello! I am your first aid assistant. Type 'quit' to exit.
